In [1]:
import ipywidgets
from IPython.display import display
import jetson.inference
import jetson.utils
import cv2

# turn numpy array into a jpeg
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])


# load detect net
net = jetson.inference.detectNet("ssd-mobilenet-v2", threshold=0.5)

camera = jetson.utils.gstCamera(640, 480, "0")
image_widget = ipywidgets.Image(format='jpeg')
text_widget = ipywidgets.Text(value='Hello World')
display(image_widget,text_widget)


Image(value=b'', format='jpeg')

Text(value='Hello World')

In [ ]:

while 1:
    # get new image, zero copy needs to be 1 so we can access memory from cpu
    img, width, height = camera.CaptureRGBA(zeroCopy=1)
    
    # detect, also appends image with found objects
    detections = net.Detect(img, width, height)
    
    # we need a jpeg to display, depth is 4 because of alpha channel
    image = jetson.utils.cudaToNumpy(img,width, height, 4)[:, :, [2, 1, 0]]
    #img_orig = jetson.utils.cudaToNumpy(img, width, heigth, 4)[:, :, [2, 1, 0]]

      
    image_widget.value = bgr8_to_jpeg(image)
    
    
    # lets put the found things in a text field just because
    text_widget.value = " ".join([net.GetClassDesc(d.ClassID) for d in detections])